# RDFLib Tutorial 

- 작성자: 박하람
- 작성일자: 2022. 09. 05
- 수정일자: 2023. 12. 04
- 내용: 파이썬 라이브러리 RDFLib를 활용해 데이터를 RDF 형식으로 변환하기
- 참고: [RDFLib를 사용해 데이터세트의 메타데이터를 DCAT으로 표현하기 1](https://www.blog.harampark.com/blog/rdflib-tutorial-dcat-1), [RDFLib를 사용해 데이터세트의 메타데이터를 DCAT으로 표현하기 2](https://www.blog.harampark.com/blog/rdflib-tutorial-dcat-2)

### 데이터세트의 메타데이터

- 공공데이터포털의 파일데이터 중 가장 활용도가 높은 데이터세트는 '한국고전번역원_한국문집총간_월고집(月皐集)'으로, '파일데이터 정보'에서 해당 데이터세트에 대한 메타데이터를 제공함
    - 예: 파일데이터명, 분류체계, 제공기관, 관리부서명, 관리부서 전화번호, 업데이트 주기

![월고집 데이터세트의 메타데이터](image/public-dataset-metadata.png)

- 출처: https://www.data.go.kr/data/3074298/fileData.do

### DCAT (Data Catalog Vocabulary)
- 분산된 웹 환경에서 개방된 데이터 카탈로그를 기술하기 위한 RDF 어휘로, 공공데이터 분야에서 데이터세트, 데이터 서비스의 관리와 상호운용을 위해 광범위하게 사용되고 있음
    - `dcat:Catalog`: 자원 (resource)에 대한 메타데이트를 표현하기 위한 클래스로, 데이터세트나 데이터 서비스가 포함된 집합을 기술함
    - `dcat:Dataset`: 데이터세트의 일반적인 특징을 기술하기 위한 클래스 
    - `dcat:DataService`: 데이터를 제공하는 서비스를 기술하기 위한 클래스 
    - `dcat:Distribution`: 데이터세트의 서로 다른 표현 형식 (예: JSON, CSV, XSLX)을 기술하기 위한 클래스

- 명세: https://www.w3.org/TR/vocab-dcat-2/


### 0. 모듈 설치와 불러오기

In [1]:
# rdflib 모듈을 설치하기 위한 코드
!pip install rdflib

In [12]:
# 필요한 모듈 불러오기
import numpy as np
import pandas as pd 
from tqdm import tqdm

from rdflib import Namespace, Literal, URIRef
from rdflib.graph import Graph
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD

### 1. 데이터 불러오기

- 원데이터 출처: [공공데이터포털의 공공데이터 11월 개방현황](https://www.data.go.kr/bbs/ntc/selectNotice.do?pageIndex=1&originId=NOTICE_0000000003409&atchFileId=FILE_000000002847531&nttApiYn=N&searchCondition2=2&searchKeyword1=)
    - 공공데이터포털은 해당 포털이 개방하고 있는 데이터세트의 목록을 제공하고 있음

In [3]:
data = pd.read_csv("data/개방목록현황_11월.csv", encoding="utf-8", dtype=str)
data = data.replace({np.nan : None})
data.head()

,목록키,목록유형,목록명,목록설명,조회수,분류체계,기관코드,기관명,국가중점여부,표준데이터여부,목록 등록일,목록 수정일,목록 URL,다운로드_활용건수,키워드
0,15125221,FILE,경기도 가평군_주정차위반단속위치현황,"가평군의 주정차 위반 단속 위치 현황입니다. 집계년도, 시군명, 관리기관명, 단속일...",(NULL),교통및물류 - 도로,4160000,경기도 가평군,N,N,2023-11-30,2023-11-30,https://www.data.go.kr/data/15125221/fileData.do,0,"주정차단속,단속장소,불법주차"
1,15125222,FILE,한국자활복지개발원_자활근로 참여자 최초보장구분별 현황,"시도별, 최초보장구분별 누적 자활근로 참여자 수<br/> - 최초보장구분 : 일반...",(NULL),사회복지 - 취약계층지원,B554042,한국자활복지개발원,N,N,2023-11-30,2023-11-30,https://www.data.go.kr/data/15125222/fileData.do,0,"자활,자활참여자,최초보장구분"
2,15125223,FILE,한국자활복지개발원_자활근로참여자 연령별 현황,"시도별, 연령별 누적 자활근로 참여자 수<br/> - 연령구분 : 19세이하, 2...",1,사회복지 - 취약계층지원,B554042,한국자활복지개발원,N,N,2023-11-30,2023-11-30,https://www.data.go.kr/data/15125223/fileData.do,0,"자활근로,자활근로 참여자,연령"
3,15125228,FILE,과학기술정보통신부_우주산업 위성활용 부문 매출 및 수출입액,"공공데이터포털에 기 제공되고 있는 우주산업실태조사 내 위성활용 부문 매출 및 수입,...",4,과학기술 - 과학기술연구,1721000,과학기술정보통신부,N,N,2023-11-30,2023-11-30,https://www.data.go.kr/data/15125228/fileData.do,1,"우주산업,우주산업실태조사,위성활용"
4,15125240,FILE,한국자활복지개발원_자활근로 참여자 사업유형별 현황,"시도별, 사업유형별 누적 자활근로 참여자 수<br/> - 유형 : 시장진입형사업단...",40,사회복지 - 취약계층지원,B554042,한국자활복지개발원,N,N,2023-11-30,2023-11-30,https://www.data.go.kr/data/15125240/fileData.do,0,"자활,자활근로,사업유형"


### 2. 데이터 파악하기

In [4]:
print(f"데이터의 행과 열은 {data.shape}입니다.")
print(f"데이터의 컬럼명은 {data.columns}입니다.")

데이터의 행과 열은 (77419, 15)입니다.
데이터의 컬럼명은 Index(['목록키', '목록유형', '목록명', '목록설명', '조회수', '분류체계', '기관코드', '기관명', '국가중점여부',
       '표준데이터여부', '목록 등록일', '목록 수정일', '목록 URL', '다운로드_활용건수', '키워드'],
      dtype='object')입니다.


In [5]:
print(f"고유한 목록키의 개수는 {len(data['목록키'].unique())}개입니다.")

고유한 목록키의 개수는 77419개입니다.


In [6]:
data.groupby(["국가중점여부"])['목록키'].count()

국가중점여부
N    74480
Y     2939
Name: 목록키, dtype: int64

In [7]:
data.groupby(["표준데이터여부"])["목록키"].count()

표준데이터여부
N    77216
Y      203
Name: 목록키, dtype: int64

In [8]:
data = data.drop(["목록유형", "조회수", "분류체계"], axis=1)

In [9]:
data[data.columns] = data.apply(lambda x: x.str.strip())

### 3. 데이터 변환하기

#### 변환을 위한 함수 

In [10]:
# namespace 
koor_def = "http://vocab.datahub.kr/def/organization/"
koor_id = "http://data.datahub.kr/id/organization/"
KOOR = Namespace(koor_def)

dcat_id = "http://data.datahub.kr/id/dcat/"

# function (convert data to rdf)
def cell(store, s, p, df_col, datatype = None, lang = None):
    if df_col != None:
        store.add((s, p, Literal(df_col, datatype=datatype, lang = lang)))
        
def uri(store, s, p, df_col, objClass = None, objURI = None) :
    if df_col != None :
        obj = URIRef(objURI + df_col) 
        store.add((s, p, obj))
        if objClass != None :
            store.add((obj, RDF.type, objClass))

In [11]:
%%time

# generate Graph()
g = Graph()
g.bind("koor", KOOR)
g.bind("dcat", DCAT)
g.bind("dct", DCTERMS)

for idx, row in tqdm(data.iterrows(), total=data.shape[0]):
    # base id 
    cat_core_uri = URIRef(dcat_id + "national-cord-data-catalog")
    cat_stan_uri = URIRef(dcat_id + "standard-data-catalog")
    ds_uri = URIRef(dcat_id + "ds-" +row["목록키"])
    dist_uri = URIRef(dcat_id + "dt-" + row["목록키"])
    orga_uri = URIRef(koor_id + row["기관코드"])

    # define uri type
    g.add((cat_core_uri, RDF.type, DCAT.Catalog))
    g.add((cat_stan_uri, RDF.type, DCAT.Catalog))
    g.add((ds_uri, RDF.type, DCAT.Dataset))
    g.add((dist_uri, RDF.type, DCAT.Distribution))
    g.add((ds_uri, DCAT.distribution, dist_uri))

    # dataset in catalog
    if row["국가중점여부"] == "Y":
        g.add((cat_core_uri, DCAT.dataset, ds_uri))
    
    if row["표준데이터여부"] == "Y":
        g.add((cat_stan_uri, DCAT.dataset, ds_uri))

    # catalog metadata
    cell(g, cat_core_uri, DCTERMS.title, "국가중점데이터 목록")
    cell(g, cat_stan_uri, DCTERMS.title, "표준데이터 목록")

    # dataset metadata
    cell(g, ds_uri, DCTERMS.title, row["목록명"], lang="ko")
    cell(g, ds_uri, DCTERMS.description, row["목록설명"], lang="ko")
    uri(g, ds_uri, DCTERMS.publisher, row["기관코드"], objClass=KOOR.Organization, objURI=koor_id)
    cell(g, orga_uri, RDFS.label, row["기관명"], lang="ko")
    cell(g, ds_uri, DCTERMS.issued, row["목록 등록일"], datatype=XSD.date)
    cell(g, ds_uri, DCTERMS.modified, row["목록 수정일"], datatype=XSD.date)
    cell(g, dist_uri, DCAT.accessURL, row["목록 URL"], datatype=XSD.anyURI)

# the number of triples
print(f"총 {len(g)} 개의 트리플이 있습니다.")

# save as ttl
g.serialize(destination=f"data/data-go-kr-metadata-dcat.ttl", format="ttl")

100%|██████████| 77419/77419 [00:14<00:00, 5312.87it/s]


총 701959 개의 트리플이 있습니다.
CPU times: user 33.6 s, sys: 570 ms, total: 34.1 s
Wall time: 34.7 s


<Graph identifier=Nea0038eac73d4815b12b6c710fd81f47 (<class 'rdflib.graph.Graph'>)>

### 4. SPARQL 질의하기

In [115]:
# 데이터세트 개수 구하기
query = """
SELECT (COUNT(DISTINCT ?dataset) AS ?dataset_count)
WHERE { 
    ?dataset a dcat:Dataset .
}
"""
result = g.query(query)
for row in result:
    print(f"고유한 데이터세트의 개수는 {row.dataset_count}개입니다.")

고유한 데이터세트의 개수는 63468개입니다.


In [116]:
# 표준데이터 카탈로그와 국가중점데이터 카탈로그에 속한 데이터세트 개수 구하기
query = """
SELECT ?title (COUNT(DISTINCT ?dataset) AS ?dataset_count)
WHERE { 
    ?catalog a dcat:Catalog ;
        dcat:dataset ?dataset ;
        dct:title ?title .
} GROUP BY ?catalog
"""
result = g.query(query)
for row in result:
    print(f"{row.title}의 데이터세트 개수는 {row.dataset_count}개입니다.")

표준데이터 목록의 데이터세트 개수는 147개입니다.
국가중점데이터 목록의 데이터세트 개수는 2847개입니다.


In [118]:
# 데이터세트의 메타데이터 구하기
query = """
SELECT DISTINCT ?dataset ?title ?orgName ?issued ?modified ?accessURL
WHERE { 
    ?dataset a dcat:Dataset ;
        dct:title ?title ;
        dct:publisher ?orgURI ;
        dct:issued ?issued ;
        dct:modified ?modified ;
        dcat:distribution ?distribution .
    ?orgURI rdfs:label ?orgName .
    ?distribution dcat:accessURL ?accessURL .
} LIMIT 10
"""
result = g.query(query)
for row in result:
    print(row.dataset, row.title, row.orgName, row.issued, row.modified, row.accessURL)

http://data.datahub.kr/id/dcat/ds-15087138 한국법제연구원_세계법령정보사이트DB 한국법제연구원 2021-09-02 2021-09-03 https://www.data.go.kr/data/15087138/fileData.do
http://data.datahub.kr/id/dcat/ds-15033949 한국남부발전(주)_설계기술용역 시공도급계약 현황(삼척) 한국남부발전(주) 2021-03-19 2022-03-07 https://www.data.go.kr/data/15033949/fileData.do
http://data.datahub.kr/id/dcat/ds-15086818 대구광역시_(비정형데이터)2021년 대구시 화보집4 대구광역시 2021-08-30 2021-08-30 https://www.data.go.kr/data/15086818/fileData.do
http://data.datahub.kr/id/dcat/ds-15064648 건강보험심사평가원_보건의료빅데이터개방시스템_의료급여진료통계 건강보험심사평가원 2020-09-15 2021-09-22 https://www.data.go.kr/data/15064648/fileData.do
http://data.datahub.kr/id/dcat/ds-15090954 강원도 원주시_원주시청홈페이지 관광포털메뉴 강원도 원주시 2021-09-29 2021-09-29 https://www.data.go.kr/data/15090954/fileData.do
http://data.datahub.kr/id/dcat/ds-15039951 인천광역시 서구_식품접객업소 인천광역시 서구 2019-09-26 2021-09-09 https://www.data.go.kr/data/15039951/fileData.do
http://data.datahub.kr/id/dcat/ds-15099429 경상남도 의령군_약국현황 경상남도 의령군 2022-03-18 2022-03-18 https://www.data.go.kr/d

In [120]:
# 데이터세트를 제공하는 상위 10개 기관 구하기
query = """
SELECT ?orgName (COUNT(DISTINCT ?dataset) AS ?dataset_count)
WHERE { 
    ?dataset a dcat:Dataset ;
        dct:publisher ?org .
    ?org rdfs:label ?orgName .
} GROUP BY ?org
ORDER BY DESC(?dataset_count)
LIMIT 10
"""
result = g.query(query)
for row in result:
    print(f"{row.orgName}의 데이터세트 개수는 {row.dataset_count}개입니다.")

서울특별시의 데이터세트 개수는 2460개입니다.
제주특별자치도의 데이터세트 개수는 1289개입니다.
동북아역사재단의 데이터세트 개수는 1028개입니다.
경기도의 데이터세트 개수는 1015개입니다.
대전광역시의 데이터세트 개수는 991개입니다.
국토교통부의 데이터세트 개수는 955개입니다.
인천광역시의 데이터세트 개수는 780개입니다.
행정안전부의 데이터세트 개수는 779개입니다.
경상남도의 데이터세트 개수는 758개입니다.
국가철도공단의 데이터세트 개수는 749개입니다.
